# Pitcher K Dataset

Assemble the dataset used to train pitcher k for a single game

#### Improvements that can be made
- Don't use opponent previous season for current predictions. Especially later in the season, we should have better data.
- Pitcher handedness, left handed batters in lineup, right handed batters in lineup

In [130]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [131]:
# Load the datasets
pitcher_physical = pd.read_csv("../../data/processed/pitcher_physical.csv", parse_dates=["birth_date"])
pitcher_game_logs = pd.read_csv("../../data/processed/pitcher_game_logs.csv", parse_dates=["game_date"])
batter_game_logs = pd.read_csv("../../data/processed/batter_game_logs.csv", parse_dates=["game_date"])
weather_data = pd.read_csv("../../data/historical/game_weather.csv")

In [132]:
# ===============================
# CREATE THE BASE MATCHUPS TABLE
# ===============================

# Create the pitcher matchups table.
pitcher_matchups = pitcher_game_logs[[
    "game_pk", "pitcher", "game_date", "team", "opp", "pitcher_handedness", 'is_starter'
]].drop_duplicates()

In [133]:
print("Pitcher matchups:", pitcher_matchups.shape)

Pitcher matchups: (67241, 7)


In [134]:
# ===============================
# BATTERS FACED AND PITCHES THROWN
# ===============================

# Aggregate per pitcher per game_date
workload_trailing = (
    pitcher_game_logs.groupby(['pitcher', 'game_date'], as_index=False)
    .agg(
        total_pitches_game=('total_pitches', 'sum'),
        batters_faced_game=('batters_faced', 'sum'),
    )
)

# Sort by pitcher/date
workload_trailing = workload_trailing.sort_values(['pitcher', 'game_date'])

# Compute trailing stats up *through* this game
workload_trailing['pit_trailing_5_bf_avg'] = (
    workload_trailing
    .groupby('pitcher')['batters_faced_game']
    .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)

workload_trailing['pit_trailing_5_pitches_avg'] = (
    workload_trailing
    .groupby('pitcher')['total_pitches_game']
    .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)


# Shift the GAME DATE *back* by 1 so these stats line up with the NEXT GAME
workload_trailing['date'] = (
    workload_trailing
    .groupby('pitcher')['game_date']
    .shift(-1)
)

# Optional: drop intermediate columns if not needed
workload_trailing = workload_trailing[['pitcher', 'date', 'pit_trailing_5_bf_avg', 'pit_trailing_5_pitches_avg']]


In [135]:
print("Workload trailing df", workload_trailing.shape)

Workload trailing df (67206, 4)


In [136]:
# ===============================
# BUILD TRAILING PITCHER STATS TABLE
# ===============================

# Pivot per-pitcher per-hand splits
pitcher_team = pitcher_game_logs[["game_pk", "pitcher", "team"]].drop_duplicates()

pitcher_pivoted = pitcher_game_logs.pivot_table(
    index=["game_pk", "pitcher", "game_date"],
    columns="batter_stance",
    values=["total_pitches", "batters_faced", "strikeouts", "walks",
            "called_strikes", "swinging_strikes", "swings", "contact_swings"],
    aggfunc="sum",
    fill_value=0
).reset_index()

# Flatten only multi-indexed columns
if isinstance(pitcher_pivoted.columns, pd.MultiIndex):
    pitcher_pivoted.columns = [
        f"{col[0]}_vs_{col[1].lower()}" if col[1] in ["L", "R"] else col[0]
        for col in pitcher_pivoted.columns
    ]
# Rename pitcher_team columns to have _team suffix before merge
pitcher_team = pitcher_team.rename(columns={
    "game_pk": "game_pk_team",
    "team": "team_team"
})

# Merge with renamed columns
pitcher_pivoted = pitcher_pivoted.merge(
    pitcher_team,
    left_on=["game_pk", "pitcher"],
    right_on=["game_pk_team", "pitcher"]
)

# Drop the extra team column
pitcher_pivoted = pitcher_pivoted.drop(columns=["game_pk_team", "team_team"])

# Calculate raw rates
pitcher_pivoted["k_perc_vs_lhh"] = pitcher_pivoted["strikeouts_vs_l"] / pitcher_pivoted["batters_faced_vs_l"]
pitcher_pivoted["k_perc_vs_rhh"] = pitcher_pivoted["strikeouts_vs_r"] / pitcher_pivoted["batters_faced_vs_r"]
pitcher_pivoted["csw_perc_vs_lhh"] = (pitcher_pivoted["called_strikes_vs_l"] + pitcher_pivoted["swinging_strikes_vs_l"]) / pitcher_pivoted["total_pitches_vs_l"]
pitcher_pivoted["csw_perc_vs_rhh"] = (pitcher_pivoted["called_strikes_vs_r"] + pitcher_pivoted["swinging_strikes_vs_r"]) / pitcher_pivoted["total_pitches_vs_r"]

# EWMA
pitcher_pivoted = pitcher_pivoted.sort_values(["pitcher", "game_date"])
for col in ["k_perc_vs_lhh", "k_perc_vs_rhh", "csw_perc_vs_lhh", "csw_perc_vs_rhh"]:
    pitcher_pivoted["pit_ewma_" + col] = (
        pitcher_pivoted.groupby("pitcher")[col]
        .transform(lambda x: x.ewm(halflife=3, adjust=False).mean().shift(1))
    )

# Final trailing stats table
# Shift the values such that the 

# Sort by pitcher and game_date
pitcher_pivoted = pitcher_pivoted.sort_values(["pitcher", "game_date"])

# Shift all the columns up by one. This will make it so the current row's date is actually the date for the NEXT game
# We need this because if we just merge on date without shifting, it will include the OUTCOME of that game in the prediction for that same game (leakage)
pitcher_pivoted["date"] = (
    pitcher_pivoted.groupby("pitcher")["game_date"]
    .shift(-1)
)

# Final trailing stats table
pitcher_trailing_stats = pitcher_pivoted[[
    "pitcher", "date",
    "pit_ewma_k_perc_vs_lhh", "pit_ewma_k_perc_vs_rhh",
    "pit_ewma_csw_perc_vs_lhh", "pit_ewma_csw_perc_vs_rhh"
]].copy().reset_index(drop=True)


In [137]:
print("Pitcher trailing stats:", pitcher_trailing_stats.shape)

Pitcher trailing stats: (67241, 6)


In [138]:
# ===============================
# BUILD TRAILING OPPONENT STATS TABLE
# ===============================

# ------ K% EWMA -------
team_k_by_hand = (
    batter_game_logs.groupby(["game_date", "team", "pitcher_handedness"])
    .agg(
        strikeouts=("strikeouts", "sum"),
        plate_appearances=("plate_appearances", "sum")
    )
    .reset_index()
)

team_k_by_hand["k_perc"] = team_k_by_hand["strikeouts"] / team_k_by_hand["plate_appearances"]

team_k_by_hand = team_k_by_hand.sort_values("game_date")
team_k_by_hand["ewma_k_perc"] = (
    team_k_by_hand
    .groupby(["team", "pitcher_handedness"])["k_perc"]
    .transform(lambda x: x.ewm(halflife=7, adjust=False).mean())
)

# Add next game date for proper alignment
team_k_by_hand["date"] = (
    team_k_by_hand
    .groupby(["team", "pitcher_handedness"])["game_date"]
    .shift(-1)
)

# Pivot to wide format
k_ewma = team_k_by_hand.pivot(index=["date", "team"], columns="pitcher_handedness", values="ewma_k_perc").reset_index()
k_ewma.columns.name = None
k_ewma = k_ewma.rename(columns={"L": "opp_ewma_k_perc_vs_lhp", "R": "opp_ewma_k_perc_vs_rhp"})


# ------ CSW% EWMA -------
team_csw_by_hand = (
    batter_game_logs.groupby(["game_date", "team", "pitcher_handedness"])
    .agg(
        swinging_strikes=("swinging_strikes", "sum"),
        called_strikes=("called_strikes", "sum"),
        total_pitches_seen=("total_pitches_seen", "sum")
    )
    .reset_index()
)

team_csw_by_hand["csw_perc"] = (
    (team_csw_by_hand["swinging_strikes"] + team_csw_by_hand["called_strikes"]) /
    team_csw_by_hand["total_pitches_seen"]
)

team_csw_by_hand = team_csw_by_hand.sort_values("game_date")
team_csw_by_hand["ewma_csw_perc"] = (
    team_csw_by_hand
    .groupby(["team", "pitcher_handedness"])["csw_perc"]
    .transform(lambda x: x.ewm(halflife=3, adjust=False).mean())
)

# Align by actual next date
team_csw_by_hand["date"] = (
    team_csw_by_hand
    .groupby(["team", "pitcher_handedness"])["game_date"]
    .shift(-1)
)

# Pivot to wide format
csw_ewma = team_csw_by_hand.pivot(index=["date", "team"], columns="pitcher_handedness", values="ewma_csw_perc").reset_index()
csw_ewma.columns.name = None
csw_ewma = csw_ewma.rename(columns={"L": "opp_ewma_csw_perc_vs_lhp", "R": "opp_ewma_csw_perc_vs_rhp"})

# ------ Contact% EWMA -------
team_contact_by_hand = (
    batter_game_logs.groupby(["game_date", "team", "pitcher_handedness"])
    .agg(
        contact_swings=("contact_swings", "sum"),
        swings=("swings", "sum")
    )
    .reset_index()
)

team_contact_by_hand["contact_perc"] = (
    team_contact_by_hand["contact_swings"] / 
    team_contact_by_hand["swings"]
)

team_contact_by_hand = team_contact_by_hand.sort_values("game_date")
team_contact_by_hand["ewma_contact_perc"] = (
    team_contact_by_hand
    .groupby(["team", "pitcher_handedness"])["contact_perc"]
    .transform(lambda x: x.ewm(halflife=3, adjust=False).mean())
)

# Align by actual next date
team_contact_by_hand["date"] = (
    team_contact_by_hand
    .groupby(["team", "pitcher_handedness"])["game_date"]
    .shift(-1)
)

# Pivot to wide format
contact_ewma = team_contact_by_hand.pivot(index=["date", "team"], columns="pitcher_handedness", values="ewma_contact_perc").reset_index()
contact_ewma.columns.name = None
contact_ewma = contact_ewma.rename(columns={"L": "opp_ewma_contact_perc_vs_lhp", "R": "opp_ewma_contact_perc_vs_rhp"})

# ------ Final Opponent Trailing Stats ------
opponent_trailing_stats = pd.merge(k_ewma, csw_ewma, on=["date", "team"], how="outer")
opponent_trailing_stats = pd.merge(opponent_trailing_stats, contact_ewma, on=["date", "team"], how="outer")


In [139]:
print("Opponent trailing stats:", opponent_trailing_stats.shape)

Opponent trailing stats: (14912, 8)


In [140]:
# ===============================
# BUILD OUTCOMES TABLE
# ===============================

# Aggregate strikeouts per pitcher per game
starter_outcomes = (
    pitcher_game_logs
    .groupby(["game_pk", "pitcher"])
    .agg(
        strikeouts=("strikeouts", "sum"),
    )
    .reset_index()
)


In [141]:
print("Starter outcomes:", starter_outcomes.shape)


Starter outcomes: (67241, 3)


In [142]:
# ===============================
# MERGE FINAL FULL TABLE
# ===============================

# --- Merge with pitcher trailing stats ---
pitcher_trailing_stats = pitcher_trailing_stats.rename(columns={"date": "pitcher_date"})
merged_df = pitcher_matchups.merge(
    pitcher_trailing_stats,
    how="left",
    left_on=["pitcher", "game_date"],
    right_on=["pitcher", "pitcher_date"],
)
merged_df = merged_df.drop(columns=["pitcher_date"])

# --- Merge with workload trailing stats ---
workload_trailing = workload_trailing.rename(columns={"date": "workload_date"})
merged_df = merged_df.merge(
    workload_trailing,
    how="left",
    left_on=["pitcher", "game_date"],
    right_on=["pitcher", "workload_date"],
)
merged_df = merged_df.drop(columns=["workload_date"])

# --- Merge with opponent trailing stats ---
opponent_trailing_stats = opponent_trailing_stats.rename(columns={"team": "opp_team", "date": "opp_date"})
merged_df = merged_df.merge(
    opponent_trailing_stats,
    how="left",
    left_on=["opp", "game_date"],
    right_on=["opp_team", "opp_date"],
)
merged_df = merged_df.drop(columns=["opp_team", "opp_date"])

# --- Merge with weather ---
merged_df = merged_df.merge(
    weather_data,
    how='left',
    on='game_pk'
)

# --- Merge with starter outcomes ---
merged_df = merged_df.merge(
    starter_outcomes,
    how='inner',
    on=['game_pk', 'pitcher']
)

In [143]:
print("Merged DF:", merged_df.shape)


Merged DF: (67311, 24)


In [150]:
# Create a clean copy of the merged dataframe
cleaned_df = merged_df.copy()

# Drop rows with NaN values in key columns
cleaned_df = cleaned_df.dropna(subset=[
    'pit_ewma_k_perc_vs_lhh',
    'pit_ewma_k_perc_vs_rhh', 
    'pit_ewma_csw_perc_vs_lhh',
    'pit_ewma_csw_perc_vs_rhh',
    'opp_ewma_k_perc_vs_lhp',
    'opp_ewma_k_perc_vs_rhp',
    'opp_ewma_csw_perc_vs_lhp',
    'opp_ewma_csw_perc_vs_rhp',
    'opp_ewma_contact_perc_vs_lhp',
    'opp_ewma_contact_perc_vs_rhp',
])

# Fill empty wind direction values with "No Wind"
cleaned_df['wind_direction'] = cleaned_df['wind_direction'].fillna('No Wind')
cleaned_df['temp'] = cleaned_df['temp'].fillna(70)
cleaned_df['wind_speed'] = cleaned_df['wind_speed'].fillna(0)

# Initialize label encoders
handedness_encoder = LabelEncoder()
condition_encoder = LabelEncoder()
wind_direction_encoder = LabelEncoder()

# Create copies of original columns before encoding
cleaned_df['pitcher_handedness_str'] = cleaned_df['pitcher_handedness'].copy()
cleaned_df['condition_str'] = cleaned_df['condition'].copy() 
cleaned_df['wind_direction_str'] = cleaned_df['wind_direction'].copy()

# Fit and transform the categorical columns
cleaned_df['pitcher_handedness'] = handedness_encoder.fit_transform(cleaned_df['pitcher_handedness_str'])
cleaned_df['condition'] = condition_encoder.fit_transform(cleaned_df['condition_str'])
cleaned_df['wind_direction'] = wind_direction_encoder.fit_transform(cleaned_df['wind_direction_str'])

# Drop original string columns
cleaned_df = cleaned_df.drop(columns=['pitcher_handedness_str', 'condition_str', 'wind_direction_str'])


In [151]:
print("Cleaned DF:", cleaned_df.shape)

Cleaned DF: (51485, 24)


In [152]:
cleaned_df.to_csv("../../data/training/pitcher_k_training.csv", index=False)